In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [2]:
tv = pd.read_csv('./TV.csv')

In [3]:
tv.head()

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
2,LG,4.4,38870,3443,16499,21990,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: WebOS,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate
3,OnePlus,4.3,101256,9189,16499,21999,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: Android,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
4,Xiaomi,4.3,3120,305,15499,24999,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: Android,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


#q1
frequency는 price에 가장 큰 영향을 미친다.
데이터 수집 중 frequency 값 중 일부가 picture_quality, speaker 변수에 잘못 입력되었다. frequency 데이터를 취합하여 전체 tv 중 60hz 인 tv 가 몇 대인지 구하라
frequency 는 모두 hz로 끝난다.
결측치는 분석에서 제외

In [5]:
len(tv)

666

In [13]:
len(tv[tv['Frequency'].str.contains('60 Hz')]) + len(tv[tv['Picture_quality'].str.contains('60 Hz')]) +len(tv[tv['Speaker'].str.contains('60 Hz')])

510

#q2
picture quality : HD, 4K, 8K
8K 제품군의 평가 stars 평균값과 4k 제품군의 평균 점수와의 평균값 차이의 절대값을 산출
picture quality 정보는 operating system, channel, picture quality 변수에 흩어져 있다.
결과는 반올림하여 소수점 둘째자리까지 출력

In [43]:
len(tv)

666

In [52]:
tv['Picture_quality'].unique()

array(['60 Hz Refresh Rate', 'HD Ready 1366 x 768 Pixels',
       'Ultra HD (4K) 3840 x 2160 Pixels', 'Full HD 1920 x 1080 Pixels',
       'Ultra HD (4K) Crystal 4K FE UHD (3840 x 2160) Pixels',
       'Full HD 1080 x 1920 Pixels', 'HD Ready 1366 x768 Pixels',
       'Ultra HD (4K) 3840×2160 Pixels', 'Full HD 1920x1080 Pixels',
       'Ultra HD (4K) 3860 x 2160 Pixels',
       'Ultra HD (4K) 3840X2160 Pixels', '24W Speaker Output',
       'Ultra HD (4K) 3840 × 2160 Pixels',
       'Ultra HD (4K) 3840*2160 Pixels', 'Ultra HD (4K) 3840x2160 Pixels',
       '120 Hz Refresh Rate', '100 Hz Refresh Rate',
       '20 W Speaker Output', '200 Hz Refresh Rate',
       '16 W Speaker Output', '58 Hz Refresh Rate', '40 W Speaker Output',
       '50 Hz Refresh Rate', '16W Speaker Output',
       'Full HD 1366x814 Pixels', 'Full HD 1366x785 Pixels',
       'HD Ready 1366x768 Pixels', 'Full HD 1366x770 Pixels',
       'Ultra HD (8K) 7680 x 4320 Pixels', 'Full HD 1366x796 Pixels',
       '20 Speaker Ou

In [53]:
len(tv[tv['Picture_quality'].str.contains('Pixels')==True])

543

In [50]:
len(tv[tv['Picture_quality'].str.contains('4K')==True])

328

In [54]:
len(tv[tv['Picture_quality'].str.contains('8K')==True])

1

In [55]:
len(tv[tv['Operating_system'].str.contains('Pixels') == True])

24

In [56]:
len(tv[tv['channel'].str.contains('Pixels') == True])

99

In [57]:
#겹치는 값 없음
len(tv[tv['Picture_quality'].str.contains('Pixels')==True]) + len(tv[tv['Operating_system'].str.contains('Pixels') == True]) + len(tv[tv['channel'].str.contains('Pixels') == True])

666

In [72]:
tv[tv['Picture_quality'].str.contains('4K')==True]['Stars'].isna().sum(axis = 0)

0

In [73]:
tv[tv['Picture_quality'].str.contains('8K')==True]['Stars'].isna().sum(axis = 0)

0

In [76]:
tv[tv['Operating_system'].str.contains('4K') == True]['Stars'].isna().sum(axis = 0)

0

In [75]:
tv[tv['Operating_system'].str.contains('8K') == True]['Stars'].isna().sum(axis = 0)

0

In [78]:
tv[tv['channel'].str.contains('4K') == True]['Stars'].isna().sum(axis = 0)

0

In [79]:
tv[tv['channel'].str.contains('8K') == True]['Stars'].isna().sum(axis = 0)

0

In [95]:
a = tv[tv['Picture_quality'].str.contains('4K') | tv['channel'].str.contains('4K') | tv['Operating_system'].str.contains('4K')]['Stars'].mean()

In [96]:
b= tv[tv['Picture_quality'].str.contains('8K') | tv['channel'].str.contains('8K') | tv['Operating_system'].str.contains('8K')]['Stars'].mean()

In [97]:
round(np.abs(np.subtract(a,b)),2)

0.38

#q3
기존변수와 파생변수를 생성 후 해당 변수를 독립변수로 하고 평가점수 stars 를 종속변수로 하여 random forest 분석을 통해 좋은 평가를 받는 제품의 조건을 알아보고자 한다.
변수 중요도를 확인하고, 가장 큰 값을 가진 변수를 찾아라


In [122]:
#결측치 제거
tv['Reviews'].isna().sum(axis = 0), tv['Ratings'].isna().sum(axis = 0)
tv_0 = tv[tv['Ratings'] != 0]
tv_1 = tv_0[tv_0['MRP']!=0]
tv_2 = tv_1[~tv_1['channel'].str.contains('Pixels') & ~tv_1['channel'].str.contains('Oper')]

In [127]:
tv_2['review_ratios'] = tv_2['Reviews']/tv_2['Ratings']
tv_2['sale_rate'] = tv_2['current_price']/tv_2['MRP']
tv_2['netflix'] = tv_2['channel'].str.contains('Netflix') == True
tv_2['prime_video'] = tv_2['channel'].str.contains('Prime Video') == True

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/1594596884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_2['review_ratios'] = tv_2['Reviews']/tv_2['Ratings']
/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/1594596884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_2['sale_rate'] = tv_2['current_price']/tv_2['MRP']
/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/1594596884.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [129]:
tv_2['netflix'] = tv_2['netflix'].astype(int)
tv_2['prime_video'] = tv_2['prime_video'].astype(int)

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/825523077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_2['netflix'] = tv_2['netflix'].astype(int)
/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/825523077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_2['prime_video'] = tv_2['prime_video'].astype(int)


In [130]:
tv_2['high_quality'] = tv_2['Picture_quality'].str.contains('4K|8K') == True
tv_2['high_quality'] = tv_2['high_quality'].astype(int)

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/1667785214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_2['high_quality'] = tv_2['Picture_quality'].str.contains('4K|8K') == True


In [131]:
tv_f = tv_2[['review_ratios', 'MRP','sale_rate','netflix','prime_video','high_quality','Stars']]

In [132]:
tv_f

,review_ratios,MRP,sale_rate,netflix,prime_video,high_quality,Stars
1,0.137941,21999,0.395427,1,0,False,3.8
2,0.088577,21990,0.750296,1,1,False,4.4
3,0.090750,21999,0.749989,1,1,False,4.3
4,0.097756,24999,0.619985,1,1,False,4.3
5,0.089692,22900,0.698646,1,0,False,4.4
...,...,...,...,...,...,...,...
645,0.158607,19500,0.691795,1,0,False,4.0
647,0.177431,43990,0.840873,1,1,True,4.4
655,0.146341,27990,0.821686,1,1,False,4.4
657,0.215753,31900,0.860470,1,0,False,4.4


In [133]:
X_train = tv_f[['review_ratios','MRP','sale_rate','netflix','prime_video','high_quality']]
y_train = tv_f['Stars']

In [134]:
rf = RandomForestRegressor(random_state = 123)

In [135]:
rf.fit(X_train,y_train)

RandomForestRegressor(random_state=123)

In [140]:
#rf.predict
rf.feature_importances_

array([0.31220617, 0.17383963, 0.4446912 , 0.02714087, 0.02400442,
       0.01811773])

In [141]:
#q2
gal = pd.read_csv('./galaxy_users.csv')

In [142]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

#q1. 
부가서비스를 많이 사용할 수록 제품/브랜드에 관여를 많이 한다는 가설을 세우고 분석하려한다.
고객별 부가서비스 사용 현황 통계를 산출
부가 서비스 1개 사용하는 사람은 6개 사용하는 사람보다 몇 배나 많은 지 계산하라

In [144]:
add_ser = ['OnlineSecurity','OnlineBackup', 'DeviceProtection','TechSupport', 'StreamingTV', 'StreamingMovies']

In [145]:
gal.loc[:,add_ser]

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No
1,Yes,No,Yes,No,No,No
2,Yes,Yes,No,No,No,No
3,Yes,No,Yes,Yes,No,No
4,No,No,No,No,No,No
...,...,...,...,...,...,...
7027,Yes,No,Yes,Yes,Yes,Yes
7028,No,Yes,Yes,No,Yes,Yes
7029,Yes,No,No,No,No,No
7030,No,No,No,No,No,No


In [157]:
par = ["Yes","No"]

In [162]:
len(gal['customerID'].unique())

7032

In [163]:
gal_n = gal[add_ser]

In [167]:
gal_n[gal_n.isin(par) == True]

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No
1,Yes,No,Yes,No,No,No
2,Yes,Yes,No,No,No,No
3,Yes,No,Yes,Yes,No,No
4,No,No,No,No,No,No
...,...,...,...,...,...,...
7027,Yes,No,Yes,Yes,Yes,Yes
7028,No,Yes,Yes,No,Yes,Yes
7029,Yes,No,No,No,No,No
7030,No,No,No,No,No,No


In [181]:
gal_n['count'] = gal_n.apply(func=lambda x:x=="Yes").sum(axis=1)

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/2217578548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gal_n['count'] = gal_n.apply(func=lambda x:x=="Yes").sum(axis=1)


In [188]:
round(len(gal_n[gal_n['count']==1])/len(gal_n[gal_n['count'] == 6]),1)

3.4

#q2
근속 월수 tenure
월별 청구 금액 MonthlyCharges
통신사 사용 월수
-> 피어슨 상관분석을 실시하고 그 결과 중 상관계수 절대값이 가장 높은 것을 확인


In [189]:
gal.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [190]:
gal.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [191]:
gal['ChargeMonths'] = gal['TotalCharges']/gal['MonthlyCharges']

In [195]:
round(gal[['tenure','MonthlyCharges','ChargeMonths']].corr(method='pearson').abs(),3)

,tenure,MonthlyCharges,ChargeMonths
tenure,1.000,0.247,0.999
MonthlyCharges,0.247,1.000,0.247
ChargeMonths,0.999,0.247,1.000


In [ ]:
#q3
고객 이탈 여부 churn 을 종속변수로 하는 모델을 만들어보고자 한다.
데이터 분할 후 정규화를 실시하여 로지스틱 회귀분석을 실시하고,
해당 모델을 기준점 삼아 모델을 개선하고자 한다.
주어진 독립변수를 활용하여 f1-score로 모델을 평가하시오

In [196]:
gal.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'ChargeMonths'],
      dtype='object')

In [197]:
cols = ['SeniorCitizen','Partner','Dependents','tenure','MonthlyCharges','TotalCharges','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingMovies','PaperlessBilling','Churn']

In [203]:
gal.loc[:,gal.dtypes.eq(object)] = gal.loc[:,gal.dtypes.eq(object)].applymap(lambda x: 1 if x == 'Yes' else(0 if x == 'No' else -1))

In [233]:
gal_3 = gal[cols]

In [235]:
train, test = train_test_split(gal_3,train_size=0.7, random_state=123)

In [236]:
model = MinMaxScaler().fit(train)
train_m = model.transform(train)
test_m = model.transform(test)

In [238]:
train_df = pd.DataFrame(train_m, columns = train.columns )

In [244]:
lr = LogisticRegression(random_state = 123)
lr.fit(X=train_df.iloc[:,:-1], y = train_df.iloc[:,-1])
pred = lr.predict(test_m[:,:-1])

/Users/cherylkim/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
(f1_score(y_true = arr_test_nor[:, 0], 
               y_pred = pred), 2)

In [246]:
round(f1_score(y_true = test_m[:,-1], y_pred=pred),2)

0.55

In [247]:
#q3
mob = pd.read_csv('./mobiles.csv')

In [249]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#q1
성능지표 = Rom/32 + ram/2 + (num_rear_camera + num_front_camera) + 배터리용량/1000
평균으로부터 2표준편차보다 큰 값 = > 이상치
판매지수 sales를 기준으로 이상치라고판단되는 제품을 주목받는 제품이라고 할 때, 해당 제품들의 성능지표의 평균을 산출하시오

In [254]:
mean = mob['sales'].mean()
std = mob['sales'].std()
out = mean + 2*std
out

146.5515012927322

In [255]:
mob_out = mob[mob['sales']>out]

In [257]:
mob_out['index'] = mob_out['ROM']/32 + mob_out['RAM']/2 + mob_out['num_rear_camera'] + mob_out['num_front_camera'] + mob_out['battery_capacity']/1000

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_26177/2970832007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mob_out['index'] = mob_out['ROM']/32 + mob_out['RAM']/2 + mob_out['num_rear_camera'] + mob_out['num_front_camera'] + mob_out['battery_capacity']/1000


In [258]:
round(mob_out['index'].mean(),2)

11.01

In [ ]:
#q2
sales 와 상관계수가 가장 높은 변수


In [261]:
mob_2 = mob[mob['num_rear_camera']!=1]
mob_2 = mob_2[['battery_capacity','ratings','num_of_ratings','sales_price','discount_percent','sales']]

In [275]:
mob_2.corr(method='pearson').abs().loc[:,'battery_capacity'][1:].idxmax()

'sales_price'

In [ ]:
#q3
sales 를 예측하기 위해 knn 알고리즘을 사용
이웃의 개수를 바꿔가면서 가장 성능이 좋은 모델을 확보하려 한다
rmse 를 기준으로 가장 성능이 좋은 모델을 확인하고, 해당 모델의 k를 구하라
독립변수 : 판매지수를 제외한 모든 변수
종속변수 : 판매지수

In [279]:
mob3 = pd.get_dummies(mob,columns=['screen_size'],drop_first=True)

In [289]:
mob3 = mob3.set_index('sales').reset_index()

In [290]:
train, test = train_test_split(mob3, train_size = 0.8, random_state=123)
scaler = MinMaxScaler().fit(train)
train_sc = scaler.transform(train)
test_sc = scaler.transform(test)

In [291]:
test_sc

array([[1.19438034e-02, 4.92063492e-01, 2.72727273e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.64387222e-04, 4.76190476e-02, 9.09090909e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04461719e-03, 4.92063492e-01, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.07704765e-03, 2.38095238e-01, 2.72727273e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.89918620e-03, 2.38095238e-01, 6.36363636e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.82019515e-02, 4.76190476e-02, 1.81818182e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [298]:
nei = [3,5,7,9,11]
k = nei[0]
knn = KNeighborsRegressor(n_neighbors=k)
knn.fit(X = train_sc[:,1:] ,y=train_sc[:,0])
pred = knn.predict(test_sc[:,1:])
rmse = mean_squared_error(y_true =test_sc[:,0] , y_pred=pred)**0.5
rmse

0.08186682309005447

In [300]:
rmse_list = []
for n in nei : 
    knn = KNeighborsRegressor(n_neighbors=n)
    knn.fit(X = train_sc[:,1:], y = train_sc[:,0])
    pred = knn.predict(test_sc[:,1:])
    rmse = mean_squared_error(y_true=test_sc[:,0], y_pred = pred) ** 0.5
    rmse_list = rmse_list + [rmse]

In [305]:
min(rmse_list
)

0.08186682309005447

In [307]:
pd.Series(rmse_list,index=nei).idxmin()

3

In [100]:
#q4
import pandas as pd
import numpy as np
pos = pd.read_csv('./sales_pos.csv')

In [101]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [5]:
#q1
매출액이 가장 큰 상품을 확인하고, 해당 상품을 가장 많이 구매하는 직업을 확인하라

SyntaxError: invalid syntax (4238647922.py, line 2)

In [102]:
pos.groupby('prod')['purchase'].sum().idxmax()

'P00025442'

In [103]:
pos[pos['prod']=='P00025442'].groupby(['job'])['purchase'].sum().idxmax()

4

In [ ]:
#q2
martial 여부에 따라 구매하는 물품의 종류가 많이 차이나는 지 확인하고자 한다.
신혼부부 26-35 그룹을 대상으로 각 고객의 구매물품 카테고리 개수를 산출하고 결혼여부별로 평균값의 차이를 산출하시오
결측치는 0으로 대치
결측치 포함하여 푸시오

In [104]:
pos['prod_cat2']=pos['prod_cat2'].fillna(0.0)
pos['prod_cat3']=pos['prod_cat3'].fillna(0.0)

In [105]:
pos['prod_cat'] = pos['prod_cat1'].astype(int).astype(str) + '-' + pos['prod_cat2'].astype(int).astype(str) +'-' + pos['prod_cat3'].astype(int).astype(str)

In [106]:
pos.dtypes

user           int64
prod          object
gender        object
age_group     object
job            int64
city          object
marital        int64
prod_cat1      int64
prod_cat2    float64
prod_cat3    float64
purchase       int64
prod_cat      object
dtype: object

In [107]:
len(pos[pos['user']==1]['prod_cat'].unique())

21

In [108]:
pos_age =pos[pos['age_group']=='26-35']

In [109]:
pos_2 = pos_age.groupby(['marital','user'])['prod_cat'].nunique().reset_index()

In [110]:
pos_2.groupby('marital')['prod_cat'].mean()

marital
0    41.663183
1    41.792336
Name: prod_cat, dtype: float64

In [ ]:
#q3
고객 5891명을 군집화하여 각 군집별로 마케팅전략을 수립하고자 한다. 
독립변수 성별, 나이, 직업, 도시, 결혼여부, 구매 상품 종류수, 총 구매금액
k-means 군집분석을 실시하고 7개 군집으로 분석했을 때 sihouette score를 산출하시오


In [111]:
pur_n = pos.groupby('user')['prod_cat'].nunique()

In [112]:
pur_tot = pos.groupby('user')['purchase'].sum()

In [113]:
pos3 = pos[['user','prod','gender','age_group','job','city','marital']]

In [114]:
pos3['gender'] = pos3['gender'].replace({'F':0,'M':1})
pos3['age_group']  

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_32759/4142731900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos3['gender'] = pos3['gender'].replace({'F':0,'M':1})


0          0-17
1          0-17
2          0-17
3          0-17
4           55+
          ...  
550063    51-55
550064    26-35
550065    26-35
550066      55+
550067    46-50
Name: age_group, Length: 550068, dtype: object

In [115]:
uni = pos['age_group'].unique()
uni.sort()
uni

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [116]:
ser = pd.Series(range(7), index=uni)
ser

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [117]:
pos3['age_group'] = pos3['age_group'].replace(ser)

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_32759/3325567264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos3['age_group'] = pos3['age_group'].replace(ser)


In [122]:
pos3=pos3.drop(columns='prod')

In [124]:
pos3 = pos3.drop_duplicates()

In [130]:
pos3_set = pos3.set_index('user')

In [136]:
pos3_new = pd.concat([pos3_set, pur_n],axis=1)

In [135]:
pur_n

user
1        21
2        37
3        18
4        13
5        43
       ... 
6036    123
6037     48
6038      9
6039     40
6040     60
Name: prod_cat, Length: 5891, dtype: int64

In [142]:
pos3_new_new = pd.concat([pos3_new,pur_tot],axis = 1).reset_index()

In [144]:
pos3_f = pd.get_dummies(pos3_new_new,columns=['job','city'])

In [149]:
pos3_f = pos3_f.drop(columns=['user'])

In [151]:
len(pos3_f.columns)

29

In [152]:
scaler = MinMaxScaler().fit(pos3_f)
pos3_f_sc = scaler.transform(pos3_f)

In [154]:
km = KMeans(n_clusters=7, random_state=123)

In [155]:
km.fit(pos3_f_sc)

KMeans(n_clusters=7, random_state=123)

In [ ]:
km.

In [157]:
round(silhouette_score(X=pos3_f_sc, labels=km.labels_),2)

0.18

In [202]:
#q5
card = pd.read_csv('./card_cust.csv')

In [160]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [161]:
#q1
card.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,NaN,0.000000,12.0
4,10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


In [ ]:
balancer ㅏ 많을 수록, tenure 길 수록 credit_limit 이 높을 것으로 예상
tenure 별로 balance , credit limit 간 피어슨 상관푼석을 실시하고, 이 중 가장 큰 상관계수를 구하시오

In [164]:
card.groupby('TENURE')[['BALANCE','CREDIT_LIMIT']].corr(method='pearson')

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696
       CREDIT_LIMIT  0.820696      1.000000
9.0    BALANCE       1.000000      0.085474
       CREDIT_LIMIT  0.085474      1.000000
10.0   BALANCE       1.000000      0.291482
       CREDIT_LIMIT  0.291482      1.000000
11.0   BALANCE       1.000000      0.380360
       CREDIT_LIMIT  0.380360      1.000000
12.0   BALANCE       1.000000      0.460833
       CREDIT_LIMIT  0.460833      1.000000

In [ ]:
#q2
일시불 구매 금액이 높은 고객군을 도출하려고 한다.
단계1 : 고객 id 를 제외한 모든 변수 17개에 대해 z-score standardiziation 
단계2 : 표준화된 변수들에 대해 k-means 군집 분석 수행. 이 때 군집수는 2-5개 중 k-means silhouette ㅡㄹ 통해 구한 최적의 k로 설정
단계3 : 단계 2에서 도출한 각 군집 별로 ONEOFF_PURCHASES의 평균을 계산
-> 군집 별 ONEOFF_PURCHASESㅇ의 평균 중 가장 큰 값은 얼마?

In [169]:
len(card.columns)

18

In [216]:
card2 = card.drop(columns='CUST_ID')

In [220]:
scaler = StandardScaler().fit(card2)
scaler_tr = scaler.transform(card2)


In [221]:
card2_tr_nor = pd.DataFrame(scaler_tr, columns=card2.columns)

In [222]:
card2_tr_nor = card2_tr_nor.dropna()

In [223]:
si_scores =[]
for n in range(2,6):
    km = KMeans(n_clusters=n, random_state=1234)
    km.fit(card2_tr_nor)
    sc = silhouette_score(X=card2_tr_nor,labels=km.labels_)
    si_scores = si_scores + [sc]

In [224]:
si_list = pd.Series(si_scores,index=[2,3,4,5])
k = si_list.idxmax()
k

2

In [225]:
best_model = KMeans(n_clusters=k, random_state = 1234)
best_model.fit(card2_tr_nor)

KMeans(n_clusters=2, random_state=1234)

In [226]:
scaler.inverse_transform(card2_tr_nor)

array([[4.09007490e+01, 8.18182000e-01, 9.54000000e+01, ...,
        1.39509787e+02, 0.00000000e+00, 1.20000000e+01],
       [3.20246742e+03, 9.09091000e-01, 0.00000000e+00, ...,
        1.07234022e+03, 2.22222000e-01, 1.20000000e+01],
       [2.49514886e+03, 1.00000000e+00, 7.73170000e+02, ...,
        6.27284787e+02, 0.00000000e+00, 1.20000000e+01],
       ...,
       [2.28506873e+03, 1.00000000e+00, 0.00000000e+00, ...,
        1.00326521e+03, 0.00000000e+00, 1.20000000e+01],
       [2.92875670e+03, 1.00000000e+00, 1.60920000e+02, ...,
        1.09847983e+03, 0.00000000e+00, 1.20000000e+01],
       [1.46017858e+03, 1.00000000e+00, 9.37960000e+02, ...,
        1.32085529e+03, 0.00000000e+00, 1.20000000e+01]])

In [227]:
card2_org = scaler.inverse_transform(card2_tr_nor)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [232]:
card2_org = pd.DataFrame(card2_org,columns=card2.columns)

In [233]:
card2_org['cluster'] = best_model.labels_

In [236]:
round(card2_org.groupby('cluster')['ONEOFF_PURCHASES'].mean().max(),2)

4026.56

In [ ]:
#q3
1. 고객 아이디가 4의 배수가 아닌 데이터를 트레인셋, 4의 배수인 데이터를 테스트셋으로 분할
2. 트레인셋으로 의사결정나무 회귀모델을 학습
- 독립변수 16개, 고객 아이디, 일시불 구매 총액을 제외한 모든 변수
- 종속변수:일시불 구매총액
3. 생성된 모델을 테스트셋에 적용하여 일시불 구매 총액을 예측
3에서 얻은 예측 결과를 평가하기 위해 B를 계산한 값? B = (1/NSUM(PRED-TRUE)**2)**0.5

In [255]:
card = pd.read_csv('./card_cust.csv')

In [263]:
card["MINIMUM_PAYMENTS"] = card["MINIMUM_PAYMENTS"].fillna(card["MINIMUM_PAYMENTS"].mean())

In [264]:
train = card[card['CUST_ID']%4!=0]
test = card[card['CUST_ID']%4==0]

In [265]:
train = train.set_index('ONEOFF_PURCHASES').reset_index()
test = test.set_index('ONEOFF_PURCHASES').reset_index()

In [266]:
tree = DecisionTreeRegressor(random_state=1234)

In [268]:
tree.fit(X=train.iloc[:,2:], y=train.iloc[:,0])

DecisionTreeRegressor(random_state=1234)

In [271]:
prd = tree.predict(test.iloc[:,2:])

In [279]:
round((((prd - test.iloc[:,0])**2).sum()/248)**0.5,1)

1039.2

In [280]:
#q6
edu = pd.read_csv('./edu_enrollees.csv')

In [281]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [289]:
edu1 = edu.drop(columns=['city','company_size','company_type'])
edu2 = edu1.dropna()
edu3 = edu2[edu2['experience'] != '>20']
edu4 = edu3[edu3['experience'] != '<1']
edu4['experience'] = edu4['experience'].astype(int)
edu5 = edu4[edu4['last_new_job'] != '>4' ]
edu6 = edu5[edu5['last_new_job'] != 'never']

/var/folders/08/qfhf2lxn7wlcym8tkm5j1t640000gn/T/ipykernel_32759/685829809.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edu4['experience'] = edu4['experience'].astype(int)


In [290]:
base = edu6.copy()

In [313]:
B = pd.crosstab(base['relevant_experience'],base['target'],normalize=0)[1.0]['Has relevant experience']
A = pd.crosstab(base['relevant_experience'],base['target'],normalize=0)[1.0]['No relevant experience']

In [314]:
round(A / B,2)

1.77

In [315]:
#q2


In [320]:
base.head()

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
8,27107.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,1,46.0,1.0,train
11,23853.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,5,1,108.0,0.0,train
19,11399.0,0.827,Female,Has relevant experience,no_enrollment,Graduate,Arts,4,1,132.0,1.0,train
20,31972.0,0.843,Male,Has relevant experience,no_enrollment,Masters,STEM,11,1,68.0,0.0,train
21,19061.0,0.926,Male,Has relevant experience,no_enrollment,Masters,STEM,11,2,50.0,0.0,train


In [323]:
base_or = base.loc[:,'gender':'major_discipline']
base_dum = pd.get_dummies(base_or, columns = base_or.columns)
base_dum.columns

Index(['gender_Female', 'gender_Male', 'gender_Other',
       'relevant_experience_Has relevant experience',
       'relevant_experience_No relevant experience',
       'enrolled_university_Full time course',
       'enrolled_university_Part time course',
       'enrolled_university_no_enrollment', 'education_level_Graduate',
       'education_level_Masters', 'education_level_Phd',
       'major_discipline_Arts', 'major_discipline_Business Degree',
       'major_discipline_Humanities', 'major_discipline_No Major',
       'major_discipline_Other', 'major_discipline_STEM'],
      dtype='object')

In [325]:
base_dum_dr = base_dum.drop(columns=base_dum.columns[[2,4,7,10,16]])
base_dum_dr

,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
8,0,1,1,0,0,1,0,0,0,0,0,0
11,0,1,1,0,0,1,0,0,0,0,0,0
19,1,0,1,0,0,1,0,1,0,0,0,0
20,0,1,1,0,0,0,1,0,0,0,0,0
21,0,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19149,0,1,1,0,0,0,1,0,0,0,0,0
19150,1,0,1,0,0,1,0,0,0,0,0,0
19152,1,0,1,0,0,1,0,0,0,1,0,0
19153,0,1,0,0,0,1,0,0,0,1,0,0


In [331]:
col2 = ['city_development_index','experience','last_new_job','training_hours']
col1 = ['target','Xgrp']
df_job2 = pd.concat([base[col1 + col2], base_dum_dr],axis =1)

In [332]:
독립변수 16개 : target, Xgrp 제외
종속변수 : target
-> 상수항 intercept 포함한 로지스틱 회귀분석 수행
상수항을 제외한 나머지 변수들에 대한 odds ratio 중 가장 큰 값을 기술

,target,Xgrp,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
8,1.0,train,0.920,7,1,46.0,0,1,1,0,0,1,0,0,0,0,0,0
11,0.0,train,0.920,5,1,108.0,0,1,1,0,0,1,0,0,0,0,0,0
19,1.0,train,0.827,4,1,132.0,1,0,1,0,0,1,0,1,0,0,0,0
20,0.0,train,0.843,11,1,68.0,0,1,1,0,0,0,1,0,0,0,0,0
21,0.0,train,0.926,11,2,50.0,0,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19149,1.0,test,0.920,9,1,36.0,0,1,1,0,0,0,1,0,0,0,0,0
19150,0.0,test,0.920,10,3,23.0,1,0,1,0,0,1,0,0,0,0,0,0
19152,0.0,test,0.920,7,1,25.0,1,0,1,0,0,1,0,0,0,1,0,0
19153,1.0,test,0.878,14,1,42.0,0,1,0,0,0,1,0,0,0,1,0,0


In [336]:
lr = LogisticRegression(C=100000,max_iter=1000, solver='liblinear',random_state=123,fit_intercept=True)

In [337]:
lr.fit(X=df_job2.iloc[:,2:],y=df_job2.iloc[:,0])

LogisticRegression(C=100000, max_iter=1000, random_state=123,
                   solver='liblinear')

In [339]:
lr.coef_

array([[-6.17027459e+00, -2.86947767e-02,  9.40010959e-02,
        -9.42300801e-04, -2.17187479e-01, -1.86998706e-01,
        -7.69008071e-01,  5.13270671e-01, -2.90445589e-01,
         2.22930314e-01, -7.78751713e-02,  2.84976114e-01,
         1.17005475e-01,  2.46530336e-01,  4.01781704e-01,
        -4.42323720e-01]])

In [340]:
lr.intercept_

array([4.56272809])

In [341]:
pd.DataFrame(lr.coef_,columns=df_job2.columns[2:])

,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,-6.170275,-0.028695,0.094001,-0.000942,-0.217187,-0.186999,-0.769008,0.513271,-0.290446,0.22293,-0.077875,0.284976,0.117005,0.24653,0.401782,-0.442324


In [343]:
round(np.exp(lr.coef_).max(), 4) # 1.67

1.6707

In [ ]:
#q3

In [345]:
train = df_job2.loc[df_job2['Xgrp']=='train',]
test = df_job2.loc[df_job2['Xgrp']=='test',]

In [ ]:
종속변수 : target
독립변수 : 16개
knn학습 -> euclidean 거리 기준 가장 가까운 5개 데이터의 target 활용하여 예측

In [346]:
knn = KNeighborsClassifier(n_neighbors=5)

In [348]:
knn.fit(X = train.drop(columns=['target','Xgrp']), y=train['target'])

KNeighborsClassifier()

In [350]:
pred = knn.predict(test.drop(columns=['target','Xgrp']))

/Users/cherylkim/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [353]:
df_tab = pd.crosstab(test['target'],pred)

In [ ]:
round(df_tab.values.diagonal().sum() / df_tab.values.sum(), 2)

In [355]:
df_tab

col_0,0.0,1.0
target,,
0.0,1897,195
1.0,616,108


In [357]:
df_tab.values.diagonal().sum()

2005